# Problem z niemiecką wiki

Dostałem pytanie o następujący problem z niemiecką wiki:

![](images/de_wiki_problem.png)

Coś wyglądało bardzo dziwnie, postarajmy się zbadać problem.

In [1]:
import pandas as pd

columns = ['prev', 'curr', 'type', 'n']
wiki_de = pd.read_csv(f"../data/clickstream-dewiki-2022-03.tsv", delim_whitespace=True, names=columns)

wiki_de.groupby("prev")['n'].sum()


prev
\tAnführungszeichen\tlink\t58\nother-empty\tAnführungszeichen\texternal\t3869\nAmerican_Standard_Code_for_Information_Interchange\tAnführungszeichen\tlink\t19\nAusrufezeichen\tAnführungszeichen\tlink\t13\nother-search\tCastello_di_Manfredonia\texternal\t11\nother-search\tNafla\texternal\t10\nother-external\tHerrad_Schenk\texternal\t11\nother-search\tHerrad_Schenk\texternal\t49\nother-search\tLillebi\texternal\t40\nPrognose\tRetrodiktion\tlink\t16\nother-search\tRetrodiktion\texternal\t35\nother-empty\tRetrodiktion\texternal\t19\nother-search\tNadine_Kegele\texternal\t10\nother-search\tHans_Engel_(Komponist)\texternal\t13\nother-empty\tHans_Engel_(Komponist)\texternal\t44\nother-internal\tPositionsschalter\texternal\t43\nother-search\tPositionsschalter\texternal\t602\nother-empty\tPositionsschalter\texternal\t65\nother-search\tAnton_Frisch\texternal\t11\nGodzilla_vs._Kong\tKing_Kong\tlink\t11\nKing_Kong_(2005)\tKing_Kong\tlink\t11\nDie_Rückkehr_des_King_Kong\tKing_Kong\tlink\t22\no

Coż, problem jest dokładnie taki sam, chociaż jak zjechaliśmy na sam dół to widzimy, że dostaliśmy `pd.Series`...

Tak jakby jedno hasło w kolumnie `prev` było jakieś dziwne, długie i generalnie podejrzane...
Spójrzmy na długość każdego z haseł i posortujmy je po długości malejąco.

In [2]:
wiki_de['prev'].str.len().sort_values(ascending=False).head()

2515191    233166.0
913330        190.0
2171838       190.0
1880630       190.0
4499218       190.0
Name: prev, dtype: float64

Ugh, jedno hasło ma 233 166 znaków!
Ewidentnie coś jest nie tak!
Jest to coś koło linijki 2515191 w pliku.
Sprawdźmy co w niej i dookoła jest.

![](images/weird_line.png)

Oho, pojedynczy znaczek `"`.
Zapisywanie tak plików csv to proszenie się o problemy...
Pandas domyślnie myśli, że jakaś fraza znajdzie się pomiędzy dwoma `"` i wszystko pomiędzy dwoma znakami traktuje jako jeden wyraz. Z reguły ma to sens tylko u nas znaczek `"` to po prostu znaczek `"`, a nie żaden początek frazy...

Kilka linków na temat tzw. quotingu:

- [link1](https://stackoverflow.com/questions/21147058/pandas-to-csv-output-quoting-issue)
- [link2](https://techterms.com/definition/csv#:~:text=CSV%20Format,double%20quote%20marks%20the%20end.)
- [link3](https://en.wikipedia.org/wiki/Comma-separated_values#Basic_rules)

W naszym przypadku musimy wyłączyć quoting czyli zgodnie z dokumentacją [`pd.read_csv`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) ustawić `quoting=3`.

In [3]:
wiki_de = pd.read_csv(f"../data/clickstream-dewiki-2022-03.tsv", delim_whitespace=True, names=columns, quoting=3)

wiki_de.groupby("prev")['n'].sum()


prev
"                               58
"Over_the_beach"-Fähigkeit     158
$                               10
$ick                            72
(1)_Ceres                     2588
                              ... 
₩                               39
€                              172
↔                               13
∂                               59
Ɐ                               28
Name: n, Length: 488870, dtype: int64

In [4]:
wiki_de['prev'].str.len().sort_values(ascending=False).head()

1880630    190.0
2171838    190.0
4535600    190.0
913330     190.0
1822124    182.0
Name: prev, dtype: float64

Wszystko działa tak jak powinno!